In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# Defina o caminho para o arquivo train.csv
file_path = r'C:\Users\Kkk\3D Objects\HCT\df_imputed.csv'

# Leia o arquivo CSV
df = pd.read_csv(file_path)
df2 = df.copy()
# Exiba as primeiras linhas do dataframe
#df.head()


In [3]:
# Definir os estratos
estratos = ['age_at_hct', 'ethnicity', 'race_group', 'sex_match', 'graft_type']

# Criar a amostra estratificada
df_sample = df.groupby(estratos, group_keys=False).apply(lambda x: x.sample(frac=1.0, random_state=1))

# Se a amostra for maior que 10.000 linhas, amostrar novamente para reduzir para 10.000 linhas
if len(df_sample) > 10000:
    df_sample = df_sample.sample(n=10000, random_state=1)
else:
    # Se a amostra for menor que 10.000 linhas, complementar com amostras aleatórias do DataFrame original
    df_sample = pd.concat([df_sample, df.drop(df_sample.index).sample(n=10000 - len(df_sample), random_state=1)])

# Verificar o tamanho da amostra final
print(f"Tamanho da amostra final: {len(df_sample)}")

# Salvar a amostra final em um arquivo CSV
df_sample.to_csv('amostra_estratificada_final.csv', index=False)
print("Amostra salva em 'amostra_estratificada_final.csv'")


Tamanho da amostra final: 10000
Amostra salva em 'amostra_estratificada_final.csv'
